### 1. Pull case id, plaintiff name, attorney name, and lien description for liens held by the lienholder with the name 'General Assembly'.

In [ ]:
sql = """
SELECT cases.id, plaintiffs.name, attornies.name, liens.description
FROM cases
    JOIN plaintiffs 
        ON cases.plaintiff_id = plaintiff.id
    JOIN attornies
        ON cases.attorney_id = attornies.id
    JOIN liens
        ON cases.id = liens.case_id
    JOIN lienholders
        ON liens.lienholders_id = lienholders.id
WHERE lienholders.name = "General Assembly";
"""

# df = pd.read_sql_query(sql, engine)
# df.head()

### 2. Find all unique attorneys, along with their contact info, that are assigned to one or more cases that haven’t had a case update in 72 days

In [ ]:
sql = """
SELECT DISTINCT attornies.*
FROM attornies
    JOIN cases
        ON attornies.id = cases.attorney_id
    JOIN case_updates
        ON case_updates.case_id = cases.id
WHERE case_updates.created_at <= "2018-12-11";
"""

# df = pd.read_sql_query(sql, engine)
# df.head()


# I'm unsure how to make a constant 72-day check in date. The way I set it up, it'd have to be updated every day

### 3. (optional) – Rank law firms (id, name) by number of cases associated with it

In [ ]:
sql = """
SELECT law_firms.id, law_firms.name
FROM law_firms
    JOIN party_relationships 
        ON party_relationships.lawfirm_id = law_firm.id
    JOIN attornies 
        ON attornies.id = party_relationships.attorney_id
    JOIN cases
        ON cases.attorney_id = attornies.id
GROUP BY law_firms.name
ORDER BY COUNT(cases.id) DES;     
"""

# df = pd.read_sql_query(sql, engine)
# df.head()

### 4. For cases that haven't have an update in 72 days, get each case’s status based on if the liens associated are open or closed.

In [ ]:
sql = """
SELECT liens.case_id,
    CASE
        WHEN liens.open = ALL (SELECT liens.open FROM liens WHERE liens.open = "open") 
            THEN "OPEN"
        WHEN liens.open = ALL (SELECT liens.open FROM liens WHERE liens.open = "closed") 
            THEN "SETTLED"
        ELSE "PARTIAL"
    END AS case_status
FROM liens
    JOIN cases
        ON liens.case_id = cases.id
    JOIN case_updates
        ON cases.id = case_updates.case_id
WHERE case_updates.created_at <= "2018-12-11";
"""

# df = pd.read_sql_query(sql, engine)
# df.head()